In [1]:
text = ""
with open("finetune_data.txt") as f:
    lines = f.readlines()
    for line in lines:
        text += line

In [2]:
text

'\n \n[INST]Can you provide a summary of the legal principles surrounding contract law in India?[/INST]\nIn India, contract law is primarily governed by the Indian Contract Act, 1872. Here\'s a detailed summary of the legal principles:\n \n1. **Offer and Acceptance**: A contract begins with an offer made by one party to another. The offer must be clear, definite, and communicated to the offeree. Acceptance is the assent to the terms of the offer and must be unconditional and communicated to the offeror to create a binding contract.\n \n2. **Consideration**: Consideration is something of value exchanged between parties to a contract. It can be money, goods, services, or a promise to do or refrain from doing something. For a contract to be valid, there must be a lawful consideration.\n \n3. **Intention to Create Legal Relations**: Parties to a contract must intend to create legal relations for the contract to be enforceable. Social agreements and domestic arrangements usually lack this i

In [25]:
import re
re.findall("\[INST\].*", text)

['[INST]Can you provide a summary of the legal principles surrounding contract law in India?[/INST]',
 '[INST]What are the key considerations when dealing with property disputes in Indian jurisdiction?[/INST]',
 '[INST]How does precedent influence the outcome of civil cases in Indian courts?[/INST]',
 '[INST]What steps should be taken to initiate a divorce proceeding in India?[/INST]',
 '[INST]Can you explain the concept of "Doctrine of Ultra Vires" and its application in Indian corporate law?[/INST]',
 '[INST]What are the potential consequences of non-compliance with labor laws in India for a business?[/INST]',
 '[INST]How do recent amendments to the Indian Penal Code impact criminal law in India?[/INST]',
 '[INST]Can you outline the process of filing a trademark application in India?[/INST]',
 '[INST]What factors are typically considered in determining liability in a negligence claim under Indian tort law?[/INST]',
 '[INST]How does arbitration differ from litigation in the Indian leg

In [6]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain.document_loaders import HuggingFaceDatasetLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline, Text2TextGenerationPipeline,AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# import os 
table = []
# for file in os.listdir(path="data/"):
loader = PyPDFLoader("data/IPC-BNS_Table.pdf")
data = loader.load()
table +=(data)
print(data)


[Document(page_content='BNS  IPC IMPLICATIONS OF REVISION  \nClause 2 (1): “act” as well a \nseries of acts as a single act;  Section 33: The word “act” \ndenotes  as well as series of acts as \na single act [...]  The definition in section 33 explained \nthat the term “act” also denoted a series \nof acts. The definition in clause  2(1) of \nthe BNS does not convey the same \nmeaning, and may lead to confusion as \nto whether an “act” signifies a series of \nacts. In its current form, the definition \nhas no meaning.  \nClause 2(4): “Court ” means a \nJudge who is empowered by \nlaw to act judicially alone, or a \nbody of Judges, which is \nempowered by law to act \njudicially as a body, when \nsuch Judge or body of Judges \nis acting judicially;  Section 20: The words “Court of \nJustice ” denote a Judge who is \nempowered by law to act \njudicially alone, or a body of \nJudges which is empowered by \nlaw to act judicially as a body, \nwhen such Judge or body of \nJudges is acting ju

In [8]:
data[0]

Document(page_content='BNS  IPC IMPLICATIONS OF REVISION  \nClause 2 (1): “act” as well a \nseries of acts as a single act;  Section 33: The word “act” \ndenotes  as well as series of acts as \na single act [...]  The definition in section 33 explained \nthat the term “act” also denoted a series \nof acts. The definition in clause  2(1) of \nthe BNS does not convey the same \nmeaning, and may lead to confusion as \nto whether an “act” signifies a series of \nacts. In its current form, the definition \nhas no meaning.  \nClause 2(4): “Court ” means a \nJudge who is empowered by \nlaw to act judicially alone, or a \nbody of Judges, which is \nempowered by law to act \njudicially as a body, when \nsuch Judge or body of Judges \nis acting judicially;  Section 20: The words “Court of \nJustice ” denote a Judge who is \nempowered by law to act \njudicially alone, or a body of \nJudges which is empowered by \nlaw to act judicially as a body, \nwhen such Judge or body of \nJudges is acting jud

In [3]:
print(data[0].page_content)

BNS  IPC IMPLICATIONS OF REVISION  
Clause 2 (1): “act” as well a 
series of acts as a single act;  Section 33: The word “act” 
denotes  as well as series of acts as 
a single act [...]  The definition in section 33 explained 
that the term “act” also denoted a series 
of acts. The definition in clause  2(1) of 
the BNS does not convey the same 
meaning, and may lead to confusion as 
to whether an “act” signifies a series of 
acts. In its current form, the definition 
has no meaning.  
Clause 2(4): “Court ” means a 
Judge who is empowered by 
law to act judicially alone, or a 
body of Judges, which is 
empowered by law to act 
judicially as a body, when 
such Judge or body of Judges 
is acting judicially;  Section 20: The words “Court of 
Justice ” denote a Judge who is 
empowered by law to act 
judicially alone, or a body of 
Judges which is empowered by 
law to act judicially as a body, 
when such Judge or body of 
Judges is acting judicially.  It is not clear why this change was 
ne

In [4]:
!pip install tabula-py
!pip install tabulate

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB 495.5 kB/s eta 0:00:25
   ---------------------------------------- 0.1/12.0 MB 469.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/12.0 MB 469.7 kB/s eta 0:00:26
   ---------------------------------------- 0.1/12.0 MB 383.3 kB/s eta 0:00:32
   ---------------------------------------- 0.1/12.0 MB 350.1 kB/s eta 0:00:35
   ---------------------------------------- 0.1/12.0 MB 344.8 kB/s eta 0:00:35
   ---------------------------------------- 0.1/12.0 MB 313.8 kB/s eta 0:00:38
   ---------------------------------------- 0.1/12.0 MB 313.8 kB/s eta 0:00:38
   ---------------------------------------- 0.1/12.0 MB 304.6 kB/s eta 0:00:39
    ------------

In [7]:
!pip install JPype1

   ---------------------------------------- 0.0/352.2 kB ? eta -:--:--
   --- ------------------------------------ 30.7/352.2 kB 1.4 MB/s eta 0:00:01
   ----- --------------------------------- 51.2/352.2 kB 525.1 kB/s eta 0:00:01
   --------- ----------------------------- 81.9/352.2 kB 770.8 kB/s eta 0:00:01
   -------------- ----------------------- 133.1/352.2 kB 787.7 kB/s eta 0:00:01
   ---------------- --------------------- 153.6/352.2 kB 838.4 kB/s eta 0:00:01
   -------------------------------- ------- 286.7/352.2 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  348.2/352.2 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 352.2/352.2 kB 1.0 MB/s eta 0:00:00


In [17]:
!pip install PyPDF2

   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/232.6 kB 1.3 MB/s eta 0:00:01
   -------- ------------------------------ 51.2/232.6 kB 525.1 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/232.6 kB 744.7 kB/s eta 0:00:01
   ----------------------- -------------- 143.4/232.6 kB 774.0 kB/s eta 0:00:01
   ------------------------- ------------ 153.6/232.6 kB 762.6 kB/s eta 0:00:01
   -------------------------------------- 232.6/232.6 kB 835.8 kB/s eta 0:00:00


In [1]:

from tabula import read_pdf
from tabulate import tabulate
 
# #reads table from pdf file
df = read_pdf("data/IPC-BNS_Table.pdf",pages="all") #address of pdf file
print(tabulate(df))

c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ayush\OneDrive\Desktop\digilawyer\env\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is dep

NameError: name 'data' is not defined

In [3]:
df

[  Clause 2 (1): “act” as well a\rseries of acts as a single act;  \
 0                    series of acts as a single act;               
 
    Clause 2 (1): “act” as well a  
 0                            NaN  ,
                                                   BNS  \
 0   Clause 2 (1): “act” as well a\rseries of acts ...   
 1                     series of acts as a single act;   
 2                                                 NaN   
 3                                                 NaN   
 4                                                 NaN   
 5                                                 NaN   
 6                                                 NaN   
 7                                                 NaN   
 8   Clause 2(4): “Court” means a\rJudge who is emp...   
 9   Clause 2 (9): “gender”—the\rpronoun “he” and i...   
 10                               pronoun “he” and its   
 11                        derivatives are used of any   
 12                       person,

In [10]:
df

[  Clause 2 (1): “act” as well a\rseries of acts as a single act;  \
 0                    series of acts as a single act;               
 
    Clause 2 (1): “act” as well a  
 0                            NaN  ,
                                                   BNS  \
 0   Clause 2 (1): “act” as well a\rseries of acts ...   
 1                     series of acts as a single act;   
 2                                                 NaN   
 3                                                 NaN   
 4                                                 NaN   
 5                                                 NaN   
 6                                                 NaN   
 7                                                 NaN   
 8   Clause 2(4): “Court” means a\rJudge who is emp...   
 9   Clause 2 (9): “gender”—the\rpronoun “he” and i...   
 10                               pronoun “he” and its   
 11                        derivatives are used of any   
 12                       person,

In [11]:
import pandas as pd

In [14]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ------------------------------------ --- 225.3/250.0 kB 6.9 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 3.8 MB/s eta 0:00:00


In [13]:
df = pd.read_excel("data\IPC-BNS_Table.xlsx")

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\ayush\AppData\Local\Temp\ipykernel_14324\3261506554.py:1: SyntaxWarning: invalid escape sequence '\I'
  df = pd.read_excel("data\IPC-BNS_Table.xlsx")
C:\Users\ayush\AppData\Local\Temp\ipykernel_14324\3261506554.py:1: SyntaxWarning: invalid escape sequence '\I'
  df = pd.read_excel("data\IPC-BNS_Table.xlsx")


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [16]:
df = pd.read_csv("data\IPC-BNS_Table 1(Table 1).csv" ,encoding='unicode_escape')

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\ayush\AppData\Local\Temp\ipykernel_14324\151862464.py:1: SyntaxWarning: invalid escape sequence '\I'
  df = pd.read_csv("data\IPC-BNS_Table 1(Table 1).csv" ,encoding='unicode_escape')


In [17]:
df.shape

(113, 4)

In [18]:
df.head()

,BNS,IPC,IMPLICATIONS OF REVISION,Unnamed: 3
0,Clause 2 (1): act as well a series of acts a...,Section 33: The word act denotes as well as ...,The definition in section 33 explained that th...,NaN
1,Clause 2(4): Court means a Judge who is empo...,Section 20: The words Court of Justice denot...,It is not clear why this change was necessitat...,NaN
2,Clause 2 (9): genderthe pronoun he and it...,Section 8: The pronoun he and its derivative...,"While the definition is now more inclusive, it...",NaN
3,Clause 2(15): Judge means a person who is o...,Section 19: The word Judge denotes not only ...,The change proposed by the definition in the B...,NaN
4,NaN,NaN,NaN,NaN


In [19]:
df.drop(labels="Unnamed: 3", inplace=True, axis=1)

In [20]:
df.head()

,BNS,IPC,IMPLICATIONS OF REVISION
0,Clause 2 (1): act as well a series of acts a...,Section 33: The word act denotes as well as ...,The definition in section 33 explained that th...
1,Clause 2(4): Court means a Judge who is empo...,Section 20: The words Court of Justice denot...,It is not clear why this change was necessitat...
2,Clause 2 (9): genderthe pronoun he and it...,Section 8: The pronoun he and its derivative...,"While the definition is now more inclusive, it..."
3,Clause 2(15): Judge means a person who is o...,Section 19: The word Judge denotes not only ...,The change proposed by the definition in the B...
4,NaN,NaN,NaN


In [22]:
df.dropna(inplace=True)

In [23]:
df.head()

,BNS,IPC,IMPLICATIONS OF REVISION
0,Clause 2 (1): act as well a series of acts a...,Section 33: The word act denotes as well as ...,The definition in section 33 explained that th...
1,Clause 2(4): Court means a Judge who is empo...,Section 20: The words Court of Justice denot...,It is not clear why this change was necessitat...
2,Clause 2 (9): genderthe pronoun he and it...,Section 8: The pronoun he and its derivative...,"While the definition is now more inclusive, it..."
3,Clause 2(15): Judge means a person who is o...,Section 19: The word Judge denotes not only ...,The change proposed by the definition in the B...
5,Clause 2(19): mental illness shall have the ...,No equivalent provision in the IPC.,This is an error since clause (a) of Section 2...


In [36]:
tmp_dict = {
    "text":[]
}
for index, row in df.iterrows():
    text = ""
    text +="IPC " + row["IPC"] +" changes to BNS"+ row["BNS"] + "The Implication of revision is "+row["IMPLICATIONS OF REVISION"]
    tmp_dict["text"].append(text)

In [37]:
tmp_dict

{'text': ['IPC Section 33: The word \x93act\x94 denotes as well as series of acts as a single act [...] changes to BNSClause 2 (1): \x93act\x94 as well a series of acts as a single act;The Implication of revision is The definition in section 33 explained that the term \x93act\x94 also denoted a series of acts. The definition in clause 2(1) of the BNS does not convey the same meaning, and may lead to confusion as to whether an \x93act\x94 signifies a series of acts. In its current form, the definition\nhas no meaning.',
  'IPC Section 20: The words \x93Court of Justice\x94 denote a Judge who is empowered by law to act judicially alone, or a body of Judges which is empowered by law to act judicially as a body, when such Judge or body of Judges is acting judicially. changes to BNSClause 2(4): \x93Court\x94 means a Judge who is empowered by law to act judicially alone, or a body of Judges, which is empowered by law to act judicially as a body, when such Judge or body of Judges is acting ju

In [38]:
pd.DataFrame(data=tmp_dict).to_csv("BNS-IPC_changes.csv")

In [43]:
from langchain_core.prompts import ChatPromptTemplate

system = """
Use the given context to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Ensure that every response from you is accurate, reliable, and legally sound.
Note: All information provided is sourced exclusively from Bharatiya Nyaya Sanhita (BNS) and Bhartiya Nagarik Suraksha Sanhita (BNSS). Please ensure that responses are based solely on clauses from these legal codes and do not include information from any other legal sources.
Maintain the template structure while giving the answers.
Avoid adding any extra information not directly related to BNS and BNSS.
Text formatting may be utilized for clarity and presentation purposes.

Format your output with the following format:
  Intro: [Give response like welcome to digilawyer.]
- Case Type: [Insert the determined case type based on the user's query, e.g., Dowry, Harassment, Murder, Theft, etc.]
- Legal provisions: [Provide all related clauses from Bharatiya Nyaya Sanhita (BNS) and Bhartiya Nagarik Suraksha Sanhita (BNSS) pertaining to the identified case type, with examples and applicable factors, along with any exceptions.]
- Legal Remedies: [Furnish a legal remedy based on the identified case type and relevant clauses from BNS and BNSS, offering actionable advice to the user.]
- Outro: [Give response like Thank you for using digilawyer.]

"""

human_prompt = """
Use the given context only to answer the question. If the answer is not in context just say "Sorry I don't know the answer of the given question". Don't frame the answer on your own 
context = {context}
question = {question}
"""
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human_prompt),
    ]
)
# final_prompt = PromptTemplate.from_template(template=template)
print(final_prompt.format(context="Here is context", question="Here is the question"))

System: 
Use the given context to answer the question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Ensure that every response from you is accurate, reliable, and legally sound.
Note: All information provided is sourced exclusively from Bharatiya Nyaya Sanhita (BNS) and Bhartiya Nagarik Suraksha Sanhita (BNSS). Please ensure that responses are based solely on clauses from these legal codes and do not include information from any other legal sources.
Maintain the template structure while giving the answers.
Avoid adding any extra information not directly related to BNS and BNSS.
Text formatting may be utilized for clarity and presentation purposes.

Format your output with the following format:
  Intro: [Give response like welcome to digilawyer.]
- Case Type: [Insert the determined case type based on the user's query, e.g., Dowry, Harassment, Murder, Theft, etc.]
- Legal provisions: [Provide all related clauses from Bharatiya Nyaya Sanhita 

# Test fine tune data

In [44]:
import pandas as pd

In [45]:
df = pd.read_excel("Finetuning_dataset.xlsx")

In [46]:
df.head()

,question,case_desc,case_type,provision,remedy
0,How can I ensure that the provisions of the Bh...,The person resides in a tribal area and seeks ...,Application of Law to Specific Area,Clause 1 of BNSS ( Bharatiya Nagarik Suraksha ...,The concerned individual can request the State...
1,I am facing legal proceedings that involve the...,The person is involved in legal proceedings th...,Authority of Magistrates under the Bharatiya N...,"Under the Bharatiya Nagarik Suraksha Sanhita, ...",The person should ensure that the legal procee...
2,I have been charged with an offence,The person has been charged with an offence an...,Investigation and trial procedures under the B...,Clause 4 Under Bharatiya Nagarik Suraksha Sanh...,"The person's offence will be investigated, inq..."
3,I have concerns about how the Bharatiya Nyaya ...,The person is seeking clarification on how the...,Interpretation of the impact of the Bharatiya ...,According to the Bharatiya Nagrik Suraksha San...,The person can rest assured that existing spec...
4,I need to understand the hierarchy of Criminal...,The person is seeking clarification on the hie...,Hierarchy of Criminal Courts under the Bharati...,"Under the Bharatiya Nagarik Suraksha Sanhita, ...","In the person's state, they can expect to find..."


In [68]:
def format_row(row):
    text = f"<s>[INST]{row["question"]}[/INST]Case Type:{row["case_type"]}\nCase Description:-{row["case_desc"]}\nProvisions:-{row["provision"]}\nLegal Remedies{row["remedy"]}</s>"

    return text

In [54]:
format_row(df.iloc[0])

<s>[INST]How can I ensure that the provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023 apply to my tribal area?[/INST]Case Type:Application of Law to Specific Area
Case Description:-The person resides in a tribal area and seeks to ensure that the provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023 apply to their locality.
Provisions:-Clause 1 of BNSS ( Bharatiya Nagarik Suraksha Sanhita), 
Short title,extent and commencement.
This Clause state that :
The provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023, do not automatically apply to tribal areas, including those in Assam, unless the concerned State Government chooses to apply them through a notification.
Legal RemediesThe concerned individual can request the State Government to issue a notification applying the relevant provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023 to their tribal area. The State Government has the authority to do so, with any necessary modifications, by issuing a notification in th

In [69]:
new_dic = {
    "text":[]
}
for idx in range(df.shape[0]):
    new_dic["text"].append(format_row(df.iloc[idx]))


In [70]:
new_dic

{'text': ['<s>[INST]How can I ensure that the provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023 apply to my tribal area?[/INST]Case Type:Application of Law to Specific Area\nCase Description:-The person resides in a tribal area and seeks to ensure that the provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023 apply to their locality.\nProvisions:-Clause 1 of BNSS ( Bharatiya Nagarik Suraksha Sanhita), \nShort title,extent and commencement.\nThis Clause state that :\nThe provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023, do not automatically apply to tribal areas, including those in Assam, unless the concerned State Government chooses to apply them through a notification.\nLegal RemediesThe concerned individual can request the State Government to issue a notification applying the relevant provisions of the Bharatiya Nagarik Suraksha Sanhita, 2023 to their tribal area. The State Government has the authority to do so, with any necessary modifications, by issuing a n

In [71]:
new_df = pd.DataFrame(data=new_dic)

In [72]:
new_df.head()

,text
0,<s>[INST]How can I ensure that the provisions ...
1,<s>[INST]I am facing legal proceedings that in...
2,<s>[INST]I have been charged with an offence[/...
3,<s>[INST]I have concerns about how the Bharati...
4,<s>[INST]I need to understand the hierarchy of...


In [73]:
new_df.to_csv("formated_fine_tune_data.csv", index=False)